In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
import matplotlib.pyplot as plt
import time
import warnings
warnings.simplefilter('ignore')


In [ ]:
train=pd.read_csv('../input/ventilator-pressure-prediction/train.csv', index_col='id')
test=pd.read_csv('../input/ventilator-pressure-prediction/test.csv', index_col='id')
submission=pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv', index_col='id')
print(train.shape, test.shape, submission.shape)

In [ ]:
train.isna().sum()

In [ ]:
train.head()

In [ ]:
figure, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
figure.set_size_inches(14,6)
sns.distplot(train['pressure'], fit=norm, ax=ax1)
sns.distplot(np.log(train['pressure']+1), fit=norm, ax=ax2)

In [ ]:
train=train.drop(train.loc[train['pressure']>40].index)

In [ ]:
#train['pressure']=np.log(train['pressure']+1)
Ytrain=train['pressure']

In [ ]:
Ytrain.isna().sum()

In [ ]:
data=train
train=train[list(test)]
all_data=pd.concat((train, test))
all_data.shape

In [ ]:
print(train.shape, test.shape)
Xtrain=all_data[:len(train)]
Xtest=all_data[len(train):]
print(Xtrain.shape, Ytrain.shape, Xtest.shape, submission.shape)

In [ ]:


from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#import optuna
from sklearn.model_selection import cross_val_score


In [ ]:
model_LGB=LGBMRegressor(objective='regression_l1', random_state=7014, subsample_freq=1,
                        colsample_bytree=0.3261853512759363, min_child_samples=221, n_estimators=2151, num_leaves= 45, 
                        reg_alpha=0.9113713668943361, reg_lambda=0.8220990333713991, subsample=0.49969995651550947, 
                        max_bin=202, learning_rate=0.02959820893211799) #,device='gpu')
model_LGB


In [ ]:
model_xgb=XGBRegressor(tree_method='gpu_hist', seed=18, objective='reg:linear', n_jobs=-1, verbosity=0,
                       colsample_bylevel=0.764115402027029, colsample_bynode=0.29243734009596956, 
                       colsample_bytree= 0.7095719673041723, gamma= 4.127534050725986, learning_rate= 0.02387231810322894, 
                       max_depth=14, min_child_weight=135, n_estimators=828,reg_alpha=0.3170105723222332, 
                       reg_lambda= 0.3660379465131937, subsample=0.611471430211575)
model_xgb

In [ ]:
model_Cat=CatBoostRegressor(loss_function='MAE', random_seed=7014, verbose=False,
                            iterations=2681, learning_rate=0.2127106032536721, depth=7, l2_leaf_reg=5.266150673910493, 
                            random_strength=7.3001140226199315, bagging_temperature=0.26098669708900213)
model_Cat



In [ ]:


model_Cat.fit(Xtrain, Ytrain)
model_LGB.fit(Xtrain, Ytrain)
model_xgb.fit(Xtrain, Ytrain)

#lgb_predictions=model_LGB.predict(Xtest)
cat_predictions=model_Cat.predict(Xtest)
#xgb_predictions=model_xgb.predict(Xtest)



In [ ]:
lgb_predictions=model_LGB.predict(Xtest)
cat_predictions=model_Cat.predict(Xtest)
xgb_predictions=model_xgb.predict(Xtest)

In [ ]:
predictions=(lgb_predictions + cat_predictions + xgb_predictions)/3
#predictions=(lgb_predictions + cat_predictions + xgb_predictions)/3
#predictions=np.exp(predictions)-1
submission['pressure']=predictions
submission.to_csv('Result2.csv')
submission.head()